In [7]:
import math

def calculate_angle(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    angle_rad = math.atan2(y2 - y1, x2 - x1)
    angle_deg = math.degrees(angle_rad)
    return angle_deg

In [8]:
import cv2
import numpy as np

# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    if ret:
        frame = cv2.GaussianBlur(frame, (5, 5), 0)
        
        # Convert the frame from BGR to HSV color space
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # lower boundary GREEN color range values
        lower = np.array([40, 100, 100])
        upper = np.array([70, 255, 255])
        
        mask = cv2.inRange(hsv, lower, upper)
        
        result = cv2.bitwise_and(frame, frame, mask=mask)
        
        # Find contours in the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Sort contours based on y-coordinate (top to bottom)
        contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[1])

        # Store centroids in a list
        centroids = []

        # Iterate over the contours and label them
        for i, contour in enumerate(contours, start=1):
            # Get the bounding box of the contour
            x, y, w, h = cv2.boundingRect(contour)

            centroid_x = x + w // 2
            centroid_y = y + h // 2

            # Store centroid coordinates
            centroids.append((centroid_x, centroid_y))

            # Draw a rectangle around the detected object and label
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.circle(frame, (centroid_x, centroid_y), 2, (0, 255, 255), 2)
            cv2.putText(frame, str(i), (centroid_x + 10, centroid_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        # Connect centroids with lines
        if len(centroids) >= 2:
            for i in range(len(centroids) - 1):
                cv2.line(frame, centroids[i], centroids[i + 1], (255, 0, 0), 2)
                
            # Calculate the angle between point 2 and point 3
            if len(centroids) >= 3:
                angle_point2_to_point3 = calculate_angle(centroids[1], centroids[2])
                cv2.putText(frame, str(angle_point2_to_point3), (centroids[1][0] - 50, centroids[1][1] - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Webcam Feed', frame)
    
    # cv2.imshow('result', result)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 